# **Deep transfer learning based on GAN**
---
<font color=black size=3 face=雅黑>**Envirionment**: Tensorflow = 1.14.0 Keras = 2.2.4</font>

<font color=black size=3 face=雅黑>**A schematic introduction of implementation proposed adversarial-based models**</font>

<font color=black size=2 face=雅黑>In what follows, we explain how to implement a GAN in Keras for DTL-IFD

We will train our GAN on 1D raw signal (2D frequency spectrm images) from CWRU dataset, a dataset of 10000 1024x1 monitoring time serie (10000 64x64 RGB images) belong to 10 classes.

Schematically, the designed GAN based DTL looks like this:

* A `1D_CNN (2D_CNN)` network maps vectors of shape `1024x1(64x64x3)` to extracted feature of shape `(128,1)`.
    
* A `source_classifier` network maps CNN feature `(128,1)` to a Probability output with ten categories`(10,)`.
    
* A `domain_classifier` network maps CNN feature `(128,1)` to a binary score estimating the probability that the image is real.

*Step 1* : We first train the `source_classifier` to make sure that the `1D_CNN (2D_CNN)` could extract feature for correct classfication task.

*Step 2* : We train the `domain_classifier` using examples of source and domain data along with "real"/"fake" labels,thus the `domain classifier` could tell which sample from source domain (labelled as true) or target domain (labelled as fake)

*Step 3* : We train the `1D_CNN (2D_CNN)`, at every step, we move the weights of `1D_CNN (2D_CNN)` in a direction that will make the discriminator more likely to classify the samples from target domain as "true" . I.e. we train the generator to fool the discriminator.

**By iteratively conducting step 1 to step 3, the domain shift could be covered and the classfication task from source domain to target domain could be implemented** </font>

# 1 Import basic modules

In [1]:
import numpy as np
from keras.layers import Input, Dense, Activation,Reshape, Flatten, Conv2D,Conv1D,BatchNormalization, MaxPooling2D,PReLU, Dropout,Conv1D,GlobalAveragePooling1D
from keras.models import Model
from keras.optimizers import SGD
from keras.utils import to_categorical
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.layers.advanced_activations import LeakyReLU

Using TensorFlow backend.
C:\Users\phoen\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\phoen\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\phoen\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\phoen\Anaconda3\envs\te

# 2 Load dataset

In [2]:
# Load 1D data
X_0_1D = np.load("./DE/X_0.npy").reshape((-1,1024,1))
X_1_1D = np.load("./DE/X_1.npy").reshape((-1,1024,1))
X_2_1D = np.load("./DE/X_2.npy").reshape((-1,1024,1))
X_3_1D = np.load("./DE/X_3.npy").reshape((-1,1024,1))
# Load 2D data
X_0_2D = np.load("./DE/Case1_FFT.npy").reshape((-1,64,64,3))/255-0.5
X_1_2D = np.load("./DE/Case2_FFT.npy").reshape((-1,64,64,3))/255-0.5
X_2_2D = np.load("./DE/Case3_FFT.npy").reshape((-1,64,64,3))/255-0.5
X_3_2D = np.load("./DE/Case4_FFT.npy").reshape((-1,64,64,3))/255-0.5
Y_0 = np.load("./DE/Y_0.npy")
Y_1 = np.load("./DE/Y_1.npy")
Y_2 = np.load("./DE/Y_2.npy")
Y_3 = np.load("./DE/Y_3.npy")
# define the function to output shuffled dataset 
def Shuffle(x,y,split):
    index=np.random.choice(a=len(x), size=len(x), replace=False, p=None)
    X=[]
    Y=[]
    for i in range(len(x)):
        slice_x = x[index[i]]
        slice_y = y[index[i]]
        X.append(slice_x)
        Y.append(slice_y)
    X = np.array(X)
    Y = np.array(Y)
    Y = to_categorical(Y)
    train_X = X[0:int(len(x)*split)]
    train_Y = Y[0:int(len(x)*split)]
    test_X = X[int(len(x)*split):]
    test_Y = Y[int(len(x)*split):]
    return train_X,train_Y,test_X,test_Y

# 3 GAN model construction

## 3.1 1D-GAN model

In [3]:
def GAN_1D():
    """Creates three different models, one used for source only training, two used for domain adaptation"""
    inputs = Input(shape=(1024,1)) 
    x4 = Conv1D(128,3)(inputs)
    x4 = LeakyReLU()(x4)
    x4 = Dropout(0.25)(x4)
    x4 = Conv1D(64,3)(x4)
    x4 = LeakyReLU()(x4)
    x4 = Dropout(0.25)(x4)
    x4 = Conv1D(32,3)(x4)
    x4 = LeakyReLU()(x4)
    x4 = Dropout(0.25)(x4)
    x4 = Flatten()(x4)
    x4 = Dense(128)(x4)
    x4 = LeakyReLU()(x4)
    x4 = Reshape((128,1))(x4)


    source_classifier = Conv1D(32,3, name="mo1")(x4)
    source_classifier = LeakyReLU(name="mo2")(source_classifier)
    source_classifier = Dropout(0.25)(source_classifier)
    source_classifier = Conv1D(16,3, name="mo3")(source_classifier)
    source_classifier = LeakyReLU(name="mo4")(source_classifier)
    source_classifier = Flatten()(source_classifier)
    source_classifier = Dense(64,name="mo5")(source_classifier)
    source_classifier = LeakyReLU(name="mo6")(source_classifier)
    source_classifier = Dense(10, activation='softmax', name="mo")(source_classifier)  
    

    domain_classifier = Conv1D(256,3, name="do1")(x4)
    domain_classifier = LeakyReLU(name="do2")(domain_classifier)
    domain_classifier = Dropout(0.25)(domain_classifier)
    domain_classifier = Conv1D(128,3, name="do3")(domain_classifier)
    domain_classifier = LeakyReLU(name="do4")(domain_classifier)
    domain_classifier = Dropout(0.25)(domain_classifier)
    domain_classifier = Flatten()(domain_classifier)
    domain_classifier = Dense(512, name="do5")(domain_classifier)
    domain_classifier = LeakyReLU(name="do6")(domain_classifier)
    domain_classifier = Dense(128, name="do7")(domain_classifier)
    domain_classifier = LeakyReLU(name="do8")(domain_classifier)
    domain_classifier = Dense(2, activation='softmax', name="do")(domain_classifier)

    comb_model = Model(inputs=inputs, outputs=[source_classifier, domain_classifier])
    comb_model.compile(optimizer="Adam",
              loss={'mo': 'categorical_crossentropy', 'do': 'binary_crossentropy'},
              loss_weights={'mo': 2, 'do': 1}, metrics=['accuracy'], )

    source_classification_model = Model(inputs=inputs, outputs=[source_classifier])
    source_classification_model.compile(optimizer="Adam",
              loss={'mo': 'categorical_crossentropy'}, metrics=['accuracy'], )

    domain_classification_model = Model(inputs=inputs, outputs=[domain_classifier])
    domain_classification_model.compile(optimizer="Adam",
                  loss={'do': 'binary_crossentropy'}, metrics=['accuracy'])
    
    embeddings_model = Model(inputs=inputs, outputs=[x4])
    embeddings_model.compile(optimizer="Adam",loss = 'categorical_crossentropy', metrics=['accuracy'])
                        
                        
    return comb_model, source_classification_model, domain_classification_model, embeddings_model
def train_1D(trainX, trainY, trainDX, trainDY, testX,testY,testDX,testDY, save_model_name,enable_GAN = True, n_iterations =1500):
    
    batch_size =64
    
    model, source_classification_model, domain_classification_model, embeddings_model = GAN_1D()

    y_class_dummy = np.ones((len(trainX), 2))
    y_adversarial_1 = to_categorical(np.array(([1] * batch_size + [0] * batch_size)))
    
    sample_weights_class = np.array(([1] * batch_size + [0] * batch_size))
    sample_weights_adversarial = np.ones((batch_size * 2,))

    start = 0
    temp = 0
    for i in range(n_iterations):
        stop = start + batch_size
        # # print(y_class_dummy.shape, ys.shape)
        y_adversarial_2 = to_categorical(np.array(([0] * batch_size + [1] * batch_size)))
        X0 = trainX[start: stop]
        X1 = trainDX[start: stop]
        y0 = trainY[start: stop]
        y1 = np.zeros(shape = (len(trainX),2))[start: stop]


        X_adv = np.concatenate([X0, X1])
        y_class = np.concatenate([y0, np.zeros_like(y0)])

        adv_weights = []
        for layer in model.layers:
            if (layer.name.startswith("do")):
                adv_weights.append(layer.get_weights())

        if(enable_GAN):
            # note - even though we save and append weights, the batchnorms moving means and variances
            # are not saved throught this mechanism 
            stats = model.train_on_batch(X_adv, [y_class, y_adversarial_1],
                                     sample_weight=[sample_weights_class, sample_weights_adversarial])
            ## 这里训练是用源域的标签先训练分类器，同时也训练了域分类器
            k = 0
            for layer in model.layers:
                if (layer.name.startswith("do")):
                    layer.set_weights(adv_weights[k])
                    k += 1
            ## 这里说的是将之前训练的域分类器的权重给到域分类器中
            class_weights = []
            
        
            for layer in model.layers:
                if (not layer.name.startswith("do")):
                    class_weights.append(layer.get_weights())
            ## 这里说的是将之前训练的标签分类器的权重给到源域分类器中
            ## 这步相当于说是在训练真实的域分类器，更新DO层的权重
            stats2 = domain_classification_model.train_on_batch(X_adv, [y_adversarial_2])

            k = 0
            for layer in model.layers:
                if (not layer.name.startswith("do")):
                    layer.set_weights(class_weights[k])
                    k += 1

        else:
            source_classification_model.train_on_batch(X0,y0)
        start += batch_size
        if start > len(trainX) - batch_size:
            start = 0

        #if ((i + 1) % 1 == 0):
            #y_test_hat_t = source_classification_model.predict(trainDX).argmax(1)
            #if accuracy_score(trainDY.argmax(1), y_test_hat_t)>0.95:
                #print("Found a good result at Iteration %d,target accuracy = %.3f"%(i,accuracy_score(trainDY.argmax(1), y_test_hat_t)))
                #source_classification_model.save("./models/model at Iteration %d with accuracy %.3f"%(i,accuracy_score(trainDY.argmax(1), y_test_hat_t)))
        if ((i + 1) % 25 == 0):
            # print(i, stats)
            y_test_hat_t = source_classification_model.predict(testDX).argmax(1)
            y_test_hat_s = source_classification_model.predict(testX).argmax(1)
            print("Iteration %d, source accuracy =  %.3f, target accuracy = %.3f"%(i, accuracy_score(testY.argmax(1), y_test_hat_s), accuracy_score(testDY.argmax(1), y_test_hat_t)))
        
        if ((i + 1) % 25 == 0):            
            y_test_hat = source_classification_model.predict(testDX).argmax(1)
            threshold = accuracy_score(testDY.argmax(1), y_test_hat)
            if threshold > temp:
                # Save the model and update it when testing acc has improvement
                temp = threshold
                print("save the model at Iteration %d with target accuracy = %.3f"%(i,threshold))
                source_classification_model.save("./models/"+save_model_name+".h5")
    return embeddings_model,source_classification_model

## 3.2 2D-GAN model

In [4]:
def GAN_2D():
    inputs = Input(shape=(64,64,3)) 
    x4 = Conv2D(128,(3,3))(inputs)
    x4 = LeakyReLU()(x4)
    x4 = Dropout(0.25)(x4)
    x4 = Conv2D(64,(3,3))(x4)
    x4 = LeakyReLU()(x4)
    x4 = Dropout(0.25)(x4)
    x4 = Conv2D(32,(3,3))(x4)
    x4 = LeakyReLU()(x4)
    x4 = Dropout(0.25)(x4)
    x4 = Flatten()(x4)
    x4 = Dense(128)(x4)
    x4 = LeakyReLU()(x4)
    x4 = Reshape((128,1))(x4)
    source_classifier = Conv1D(32,3, name="mo1")(x4)
    source_classifier = LeakyReLU(name="mo2")(source_classifier)
    source_classifier = Dropout(0.25)(source_classifier)
    source_classifier = Conv1D(16,3, name="mo3")(source_classifier)
    source_classifier = LeakyReLU(name="mo4")(source_classifier)
    source_classifier = Flatten()(source_classifier)
    source_classifier = Dense(64,name="mo5")(source_classifier)
    source_classifier = LeakyReLU(name="mo6")(source_classifier)
    source_classifier = Dense(10, activation='softmax', name="mo")(source_classifier)  
    

    domain_classifier = Conv1D(256,3, name="do1")(x4)
    domain_classifier = LeakyReLU(name="do2")(domain_classifier)
    domain_classifier = Dropout(0.25)(domain_classifier)
    domain_classifier = Conv1D(128,3, name="do3")(domain_classifier)
    domain_classifier = LeakyReLU(name="do4")(domain_classifier)
    domain_classifier = Dropout(0.25)(domain_classifier)
    domain_classifier = Flatten()(domain_classifier)
    domain_classifier = Dense(512, name="do5")(domain_classifier)
    domain_classifier = LeakyReLU(name="do6")(domain_classifier)
    domain_classifier = Dense(128, name="do7")(domain_classifier)
    domain_classifier = LeakyReLU(name="do8")(domain_classifier)
    domain_classifier = Dense(2, activation='softmax', name="do")(domain_classifier)

    comb_model = Model(inputs=inputs, outputs=[source_classifier, domain_classifier])
    comb_model.compile(optimizer="Adam",
              loss={'mo': 'categorical_crossentropy', 'do': 'binary_crossentropy'},
              loss_weights={'mo': 2, 'do': 1}, metrics=['accuracy'], )

    source_classification_model = Model(inputs=inputs, outputs=[source_classifier])
    source_classification_model.compile(optimizer="Adam",
              loss={'mo': 'categorical_crossentropy'}, metrics=['accuracy'], )

    domain_classification_model = Model(inputs=inputs, outputs=[domain_classifier])
    domain_classification_model.compile(optimizer="Adam",
                  loss={'do': 'binary_crossentropy'}, metrics=['accuracy'])
    
    embeddings_model = Model(inputs=inputs, outputs=[x4])
    embeddings_model.compile(optimizer="Adam",loss = 'categorical_crossentropy', metrics=['accuracy'])
                        
                        
    return comb_model, source_classification_model, domain_classification_model, embeddings_model
def train_2D(trainX, trainY, trainDX, trainDY, testX,testY,testDX,testDY, save_model_name,enable_GAN = True, n_iterations =1000):
    
    batch_size =64
    
    model, source_classification_model, domain_classification_model, embeddings_model = GAN_2D()

    y_class_dummy = np.ones((len(trainX), 2))
    y_adversarial_1 = to_categorical(np.array(([1] * batch_size + [0] * batch_size)))
    
    sample_weights_class = np.array(([1] * batch_size + [0] * batch_size))
    sample_weights_adversarial = np.ones((batch_size * 2,))

    start = 0
    temp = 0
    for i in range(n_iterations):
        stop = start + batch_size
        # # print(y_class_dummy.shape, ys.shape)
        y_adversarial_2 = to_categorical(np.array(([0] * batch_size + [1] * batch_size)))
        X0 = trainX[start: stop]
        X1 = trainDX[start: stop]
        y0 = trainY[start: stop]
        y1 = np.zeros(shape = (len(trainX),2))[start: stop]


        X_adv = np.concatenate([X0, X1])
        y_class = np.concatenate([y0, np.zeros_like(y0)])

        adv_weights = []
        for layer in model.layers:
            if (layer.name.startswith("do")):
                adv_weights.append(layer.get_weights())

        if(enable_GAN):
            # note - even though we save and append weights, the batchnorms moving means and variances
            # are not saved throught this mechanism 
            stats = model.train_on_batch(X_adv, [y_class, y_adversarial_1],
                                     sample_weight=[sample_weights_class, sample_weights_adversarial])
            ## 这里训练是用源域的标签先训练分类器，同时也训练了域分类器
            k = 0
            for layer in model.layers:
                if (layer.name.startswith("do")):
                    layer.set_weights(adv_weights[k])
                    k += 1
            ## 这里说的是将之前训练的域分类器的权重给到域分类器中
            class_weights = []
            
        
            for layer in model.layers:
                if (not layer.name.startswith("do")):
                    class_weights.append(layer.get_weights())
            ## 这里说的是将之前训练的标签分类器的权重给到源域分类器中
            ## 这步相当于说是在训练真实的域分类器，更新DO层的权重
            stats2 = domain_classification_model.train_on_batch(X_adv, [y_adversarial_2])

            k = 0
            for layer in model.layers:
                if (not layer.name.startswith("do")):
                    layer.set_weights(class_weights[k])
                    k += 1

        else:
            source_classification_model.train_on_batch(X0,y0)
        start += batch_size
        if start > len(trainX) - batch_size:
            start = 0
        # check the model performance each 100 iteration
        
        if ((i + 1) % 100 == 0):            
            y_test_hat = source_classification_model.predict(testDX).argmax(1)
            threshold = accuracy_score(testDY.argmax(1), y_test_hat)
            if threshold > temp:
                # Save the model and update it when testing acc has improvement
                temp = threshold
                print("save the model at Iteration %d with target accuracy = %.3f"%(i,threshold))
                source_classification_model.save(save_model_name+".h5")
                if i >= 1000:
                    break
            
    return embeddings_model,source_classification_model

# 4 Model training

In [5]:
trainX,trainY,testX,testY = Shuffle(X_0_2D,Y_0,split=0.8)
trainDX,trainDY,testDX,testDY = Shuffle(X_1_2D,Y_1,split=0.8)
save_model_name="Adversarial_Task1_2D"
gan_model,classifier = train_2D(trainX,trainY,trainDX,trainDY,testX,testY,testDX,testDY,save_model_name,enable_GAN = True)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
save the model at Iteration 99 with target accuracy = 0.957
save the model at Iteration 299 with target accuracy = 0.994
save the model at Iteration 499 with target accuracy = 0.995


# 5 Model testing 

In [7]:

from keras.models import load_model
model=load_model(save_model_name+".h5")

def test_model(model,testDX,testDY):
    acc=np.zeros(10,)
    for i in range(10):
        acc[i]=model.evaluate(testDX[0+(len(testDX)//10)*i:(len(testDX)//10)+(len(testDX)//10)*i],testDY[0+(len(testDX)//10)*i:(len(testDX)//10)+(len(testDX)//10)*i])[1]
    print("Mean acc: %.2f"%(np.mean(acc)*100)+"%","Prediction variance: %.2f"%(np.var(acc)*100)+"%")

test_model(model,testDX,testDY)

200/200 [==============================] - ETA:  - 0s 286us/step
Mean acc: 99.45% Prediction variance: 0.00%
